In [6]:
import netCDF4 as nc
from netCDF4 import Dataset
import numpy as np
import json
import subprocess
import os
import sys
#!{sys.executable} -m pip install PyAthena
from pyathena import connect
import pandas as pd
from netCDF4 import Dataset

In [7]:
conn = connect(s3_staging_dir='s3://esdis-ems-athena', region_name='us-west-2')
cursor=conn.cursor()

In [8]:
q_create_staging ="""
CREATE EXTERNAL TABLE sci_staging.merra2_constant_model(
FRLAKE string,
FRLAND string,
FRLANDICE string,
FROCEAN string,
PHIS string,
SGH string,
lon string, 
lat string, 
time string
)
ROW FORMAT SERDE 
  'org.apache.hadoop.hive.serde2.OpenCSVSerde' 
WITH SERDEPROPERTIES ( 
  'quoteChar'='\u0000', 
  'separatorChar'='~') 
STORED AS INPUTFORMAT 
  'org.apache.hadoop.mapred.TextInputFormat' 
OUTPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION
  's3://nasa-ems-sandbox/staging/science/merra-2/constant_model/'
"""

q_create="""
CREATE EXTERNAL TABLE sci.merra2_constant_model 
(
  FRLAKE float, 
  FRLAND float, 
  FRLANDICE float, 
  FROCEAN float, 
  PHIS float, 
  SGH float, 
  lon float, 
  lat float, 
  time string
)
ROW FORMAT SERDE 
  'org.apache.hadoop.hive.ql.io.orc.OrcSerde' 
STORED AS INPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.orc.OrcInputFormat' 
OUTPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.orc.OrcOutputFormat'
LOCATION
  's3://nasa-ems-sandbox/dl/science/merra-2/constant_model/'
TBLPROPERTIES (
  'has_encrypted_data'='false',
    'format' = 'Parquet',
    'write_compression' = 'DEFLATE'  
)
"""
q="""
insert into sci.merra2_constant_model
select 
try_cast(FRLAKE as real) FRLAKE,
try_cast(FRLAND as real) FRLAND,
try_cast(FRLANDICE as real) FRLANDICE,
try_cast(FROCEAN as real) FROCEAN,
try_cast(PHIS as real) PHIS,
try_cast(SGH as real) SGH,
try_cast(lat as real) lat,
try_cast(lon as real) lon,
time
from 
 sci_staging.merra2_constant_model
order by
 try_cast(lon as real)
"""

In [9]:
ds=Dataset("./MERRA2_101.const_2d_asm_Nx.00000000.nc4")

In [16]:
def do_it(ds, outs): # t = 1 of 8 time dimensions
    vars={}
#    ds=m[d]['ds']
    print ('time=', ds['time'][:])
#    return

# t=0 for constant model parameters files
    t=0
    for v in ds.variables:
        if v in vars:
            print ('duplicate variable: ', v, ' skipping ....')
        else:
                #print (v)
            ds.set_auto_mask(False)
            if '_FillValue' in ds[v].ncattrs():
                tmp=ds[v][t][:]
                tmp[tmp == ds[v]._FillValue] = np.nan
                vars[v]=tmp[:]
            else:
                if v == 'time' or v=='lat' or v=='lon' or v=='lev':
                    vars[v]=ds[v][:]
                else:
                    vars[v]=ds[v][t][:]
        #print (d)
        
# Normalize lat/lon


    vm_lon, vm_lat = np.meshgrid( vars['lon'], vars['lat']  )

    for lat in range(vars['lat'].size):
        print (lat, vars['FRLANDICE'].shape)
        print ('Lon/Lat/FRLANDICE/FROCEAN')
        print (vm_lon[lat][0], vm_lat[lat][0],vars['FRLANDICE'][lat][0], vars['FROCEAN'][lat][0])
    
    print (vm_lon, vm_lon[:].max())
    ctr=0
    stack_this=[]
    for v in sorted(vars):
        print (ctr, v, vars[v].shape, vm_lon.shape)
        ctr+=1
        if v == 'time':
            pass
        elif v == 'lon':
                stack_this.append(vm_lon.flatten()[:])
        elif v == 'lat':
                stack_this.append(vm_lat.flatten()[:])
        else:
                #print (ctr, 'shape=', vars[v].shape)
                stack_this.append(vars[v].flatten()[:])
        #print (stack_this)
    x=np.dstack( (stack_this) )
    fname = '/home/ec2-user/merra2_constants.tsv.gz'
#    np.savetxt(fname, x[0], fmt='%.9f~%.9f~%.9f~%.9f~%.9f~%.9f~%.9f~%.9f~%.9f~%.9f~%.9f~%.9f~%.9f~%.9f~%.9f~%.9f~%.9f~%.3f~%.1f~%.3f~%.1f', delimiter='~', newline="\n")
    np.savetxt(fname, x[0], fmt='%.7g', delimiter='~', newline="\n")
    outs.append(fname)
        
    return

In [18]:
outs=[]
do_it(ds, outs) # Create constants file.
    
stagingS3  = 's3://nasa-ems-sandbox/staging/science/merra-2/constant_model/'
f=outs[0]
rc=subprocess.run(["aws", "s3", "cp", f, stagingS3 ])

    # Execute insert into ...
cursor.execute(q)
#    print(cursor.description)
#    print(cursor.fetchall())
    
#    for f in outs:
        
    # Delete s3 staging files
#        rc=subprocess.run(["aws", "s3", "rm", 's3://nasa-ems-sandbox/staging/science/merra-2/l72_3hr/' + f.rsplit('/', 1)[-1]])
#        print ('rc:',rc)
     
    # Delete tsv.gz file(s)
#        os.remove(f)
#    break


time= [0]
0 (361, 576)
Lon/Lat/FRLANDICE/FROCEAN
-180.0 -90.0 1.0 0.0
1 (361, 576)
Lon/Lat/FRLANDICE/FROCEAN
-180.0 -89.5 1.0 0.0
2 (361, 576)
Lon/Lat/FRLANDICE/FROCEAN
-180.0 -89.0 1.0 0.0
3 (361, 576)
Lon/Lat/FRLANDICE/FROCEAN
-180.0 -88.5 1.0 0.0
4 (361, 576)
Lon/Lat/FRLANDICE/FROCEAN
-180.0 -88.0 1.0 0.0
5 (361, 576)
Lon/Lat/FRLANDICE/FROCEAN
-180.0 -87.5 1.0 0.0
6 (361, 576)
Lon/Lat/FRLANDICE/FROCEAN
-180.0 -87.0 1.0 0.0
7 (361, 576)
Lon/Lat/FRLANDICE/FROCEAN
-180.0 -86.5 1.0 0.0
8 (361, 576)
Lon/Lat/FRLANDICE/FROCEAN
-180.0 -86.0 1.0 0.0
9 (361, 576)
Lon/Lat/FRLANDICE/FROCEAN
-180.0 -85.5 1.0 0.0
10 (361, 576)
Lon/Lat/FRLANDICE/FROCEAN
-180.0 -85.0 1.0 0.0
11 (361, 576)
Lon/Lat/FRLANDICE/FROCEAN
-180.0 -84.5 1.0 0.0
12 (361, 576)
Lon/Lat/FRLANDICE/FROCEAN
-180.0 -84.0 1.0 0.0
13 (361, 576)
Lon/Lat/FRLANDICE/FROCEAN
-180.0 -83.5 1.0 0.0
14 (361, 576)
Lon/Lat/FRLANDICE/FROCEAN
-180.0 -83.0 1.0 0.0
15 (361, 576)
Lon/Lat/FRLANDICE/FROCEAN
-180.0 -82.5 1.0 0.0
16 (361, 576)
Lon/Lat/FR

In [7]:
for m in inputs:
    print (m)
    break

20210101


In [10]:
cursor.execute(q)
print(cursor.description)
print(cursor.fetchall())

[('rows', 'bigint', None, None, 19, 0, 'UNKNOWN')]
[]


In [14]:
for f in outs:
    # Delete s3 staging files
    rc=subprocess.run(["aws", "s3", "rm", 's3://nasa-ems-sandbox/staging/science/merra-2/l72_3hr/' + f.rsplit('/', 1)[-1]])
    print ('rc:',rc)
     
    # Delete tsv.gz file(s)
    #os.remove(f)

rc: CompletedProcess(args=['aws', 's3', 'rm', 's3://nasa-ems-sandbox/staging/science/merra-2/l72_3hr/merra2-3d_3hr_20210101_0.tsv.gz'], returncode=0)
rc: CompletedProcess(args=['aws', 's3', 'rm', 's3://nasa-ems-sandbox/staging/science/merra-2/l72_3hr/merra2-3d_3hr_20210101_1.tsv.gz'], returncode=0)
rc: CompletedProcess(args=['aws', 's3', 'rm', 's3://nasa-ems-sandbox/staging/science/merra-2/l72_3hr/merra2-3d_3hr_20210101_2.tsv.gz'], returncode=0)
rc: CompletedProcess(args=['aws', 's3', 'rm', 's3://nasa-ems-sandbox/staging/science/merra-2/l72_3hr/merra2-3d_3hr_20210101_3.tsv.gz'], returncode=0)
rc: CompletedProcess(args=['aws', 's3', 'rm', 's3://nasa-ems-sandbox/staging/science/merra-2/l72_3hr/merra2-3d_3hr_20210101_4.tsv.gz'], returncode=0)
rc: CompletedProcess(args=['aws', 's3', 'rm', 's3://nasa-ems-sandbox/staging/science/merra-2/l72_3hr/merra2-3d_3hr_20210101_5.tsv.gz'], returncode=0)
rc: CompletedProcess(args=['aws', 's3', 'rm', 's3://nasa-ems-sandbox/staging/science/merra-2/l72_3hr

In [3]:
ds2=Dataset("/home/ec2-user/MERRA2_400.inst3_3d_asm_Nv.20211231.nc4", "r")

In [5]:
print (ds2)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    History: Original file generated: Wed Jan 12 00:59:19 2022 GMT
    Comment: GMAO filename: d5124_m2_jan10.inst3_3d_asm_Nv.20211231.nc4
    Filename: MERRA2_400.inst3_3d_asm_Nv.20211231.nc4
    Conventions: CF-1
    Institution: NASA Global Modeling and Assimilation Office
    References: http://gmao.gsfc.nasa.gov
    Format: NetCDF-4/HDF-5
    SpatialCoverage: global
    VersionID: 5.12.4
    TemporalRange: 1980-01-01 -> 2016-12-31
    identifier_product_doi_authority: http://dx.doi.org/
    ShortName: M2I3NVASM
    GranuleID: MERRA2_400.inst3_3d_asm_Nv.20211231.nc4
    ProductionDateTime: Original file generated: Wed Jan 12 00:59:19 2022 GMT
    LongName: MERRA2 inst3_3d_asm_Nv: 3d,3-Hourly,Instantaneous,Model-Level,Assimilation,Assimilated Meteorological Fields
    Title: MERRA2 inst3_3d_asm_Nv: 3d,3-Hourly,Instantaneous,Model-Level,Assimilation,Assimilated Meteorological Fields
    Southernmost

In [5]:
v_time = nc.num2date(ds2['time'][:],ds2['time'].units, only_use_python_datetimes=True)
v_lev  = ds2['lev'][:]
v_lon  = ds2['lon'][:]
v_lat  = ds2['lat'][:]
v_CLOUD = np.around(ds2['CLOUD'][:],7)
v_DELP = np.around(ds2['DELP'][:],7 )
v_EPV = np.around(ds2['EPV'][:], 7)
v_H = np.around(ds2['H'][:], 7)
v_O3 = np.around(ds2['O3'][:], 7)
v_OMEGA = np.around(ds2['OMEGA'][:], 7)
v_PHIS = np.around(ds2['PHIS'][:], 7)
v_PL = np.around(ds2['PL'][:], 7)
v_PS = ds2['PS'][:]
v_QI = ds2['QI'][:]
v_QL = ds2['QL'][:]
v_QV = ds2['QV'][:]
v_RH = ds2['RH'][:]
v_SLP = np.around(ds2['SLP'][:],7)
v_T = ds2['T'][:]
v_U = ds2['U'][:]
v_V = ds2['V'][:]

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: WARNING: valid_range not used since it
cannot be safely cast to variable data type
  if __name__ == '__main__':


In [6]:
import datetime
print (v_time)
#print (v_time[0].replace())
#x=v_time[0].strftime("%Y-%m-%d %H:%M:%S")
#print (x, type(x))
#y=datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S")
#print (y, type(y))
#print (z, type(z))
s_time=[]
for t in v_time:
    s_time.append(t.strftime("%Y-%m-%d %H:%M:%S").replace( ' ', 'T'))

print (s_time)

[cftime.DatetimeGregorian(2021, 12, 31, 0, 0, 0, 0, has_year_zero=False)
 cftime.DatetimeGregorian(2021, 12, 31, 3, 0, 0, 0, has_year_zero=False)
 cftime.DatetimeGregorian(2021, 12, 31, 6, 0, 0, 0, has_year_zero=False)
 cftime.DatetimeGregorian(2021, 12, 31, 9, 0, 0, 0, has_year_zero=False)
 cftime.DatetimeGregorian(2021, 12, 31, 12, 0, 0, 0, has_year_zero=False)
 cftime.DatetimeGregorian(2021, 12, 31, 15, 0, 0, 0, has_year_zero=False)
 cftime.DatetimeGregorian(2021, 12, 31, 18, 0, 0, 0, has_year_zero=False)
 cftime.DatetimeGregorian(2021, 12, 31, 21, 0, 0, 0, has_year_zero=False)]
['2021-12-31T00:00:00', '2021-12-31T03:00:00', '2021-12-31T06:00:00', '2021-12-31T09:00:00', '2021-12-31T12:00:00', '2021-12-31T15:00:00', '2021-12-31T18:00:00', '2021-12-31T21:00:00']


In [7]:
#lon=[-180, -90, 0, 90, 180]
#lat=[-90, 0, 90]
#m_lat,m_lon = np.meshgrid(lat,lon)
#print (m_lon.shape, m_lon)
#print (m_lat.shape, m_lat)

In [8]:
#print ('1', np.repeat([[1,2],[3,4]], 2))
#x = np.array( [[1,2],[3,4]] )
#print ('2a', np.repeat(x, 2))
#[     C C
#R    [1,2],
#R    [3,4]
#]
#print ('2b', np.repeat(x, 5, axis=1))
#print ('2c', np.repeat(x, 5, axis=0))
#print ('3', np.repeat(x, 3, axis=1))
#print ('4', np.repeat(x, [1, 2], axis=0))
#y=np.zeros((5, 2, 2))
#print (x.shape, y.shape)
#for i in range(5):
#    y[i]=x
#print (y[0])

vn_PHIS=np.zeros((72,361,576))
vn_PS=np.zeros((72,361,576))
vn_SLP=np.zeros((72,361,576))

for i in range(72):
    vn_PHIS[i]=v_PHIS[0]
    vn_PS[i]=v_PS[0]
    vn_SLP[i]=v_SLP[0]

print (vn_PS.shape)



(72, 361, 576)


In [9]:
charar = np.chararray(100, itemsize=19)
charar[:] = s_time[0]
#print (charar)
d = np.zeros(10, dtype='datetime64[s]')
#print (d)
d[:]=v_time[0]
print (d)

['2021-12-31T00:00:00' '2021-12-31T00:00:00' '2021-12-31T00:00:00'
 '2021-12-31T00:00:00' '2021-12-31T00:00:00' '2021-12-31T00:00:00'
 '2021-12-31T00:00:00' '2021-12-31T00:00:00' '2021-12-31T00:00:00'
 '2021-12-31T00:00:00']


In [10]:
#vf_CLOUD = v_CLOUD.flatten()
#vf_H     = v_H.flatten()
#print (v_SLP.shape)
#X, Y = np.meshgrid(x*geo_var.perspective_point_height, y*geo_var.perspective_point_height)
#vm_lon, vm_lat = np.meshgrid(v_lon, v_lat)
print (vn_PHIS.shape, v_PL.shape)


vm_time=np.zeros(14971392, dtype='datetime64[s]')

vm_lat, vm_lev, vm_lon = np.meshgrid(v_lat, v_lev, v_lon)


t_size=v_time.size

for t in range(t_size):
#    print (vn_SLP[t].shape)
#    print (t, v_time[t])
#    print (vm_lat.shape, vm_lon.shape)
#    print (vn_PS.shape, vn_PS[t].shape)
    
    vm_time[:] = v_time[t]
    print (vm_time.shape)
    x=np.dstack((
        vm_lon.flatten(), 
        vm_lat.flatten(), 
        vm_lev.flatten(),
        vm_time.astype('float'),
        v_CLOUD[t].flatten(), 
        v_DELP[t].flatten(),
        v_EPV[t].flatten(),
        v_H[t].flatten(),
        v_O3[t].flatten(),
        v_OMEGA[t].flatten(),
        vn_PHIS.flatten(), 
        v_PL[t].flatten(), 
        vn_PS.flatten(), 
        v_QI[t].flatten(), 
        v_QL[t].flatten(), 
        v_QV[t].flatten(), 
        v_RH[t].flatten(), 
        vn_SLP.flatten(), 
        v_T[t].flatten(), 
        v_U[t].flatten(), 
        v_V[t].flatten()
    ))
    fname = '/home/ec2-user/test' + s_time[t].replace(':', '_') + '.tsv.gz'
    np.savetxt(fname, x[0], fmt='%.5f', delimiter='~', newline="\n")


(72, 361, 576) (8, 72, 361, 576)
(14971392,)
(14971392,)
(14971392,)
(14971392,)
(14971392,)
(14971392,)
(14971392,)
(14971392,)


In [26]:
np.savetxt('test.tsv.gz', x[0], fmt='%.5f', delimiter='~', newline="\n")

In [37]:
print (x.shape)
for i in range(x.size):
    v_x=x[0][i]
    print (i, v_x)
    #np.savetxt('test.tsv', v_x, fmt='%.5f', delimeter='~', newline="\n")
    break
print (i, x.shape)
print (x[0][0].min())


(1, 14971392, 8)
0 [-1.80000000e+02 -9.00000000e+01  1.00000000e+00  2.75777090e+04
  6.86709297e+04  9.97232578e+04  0.00000000e+00  1.00000024e+00]
0 (1, 14971392, 8)
-180.0


In [16]:
#for l in range(v_lev.size):
#    print (l, v_lev[l])
    
lon=1
lat=140
level=1
print (v_T[0][level][lat][lon])

205.59442


In [6]:
v_time = nc.num2date(ds2['time'][:],ds2['time'].units, only_use_python_datetimes=True)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: WARNING: valid_range not used since it
cannot be safely cast to variable data type
  if __name__ == '__main__':
